# Logistic Regression using La Classy

In [53]:
import { parse } from "jsr:@std/csv@0.218";
import {
  ClassificationReport,
  Matrix,
  useSplit,
  CategoricalEncoder,
} from "jsr:@lala/appraisal@0.7.5";
import {
  SagSolver,
  softmaxActivation,
  rmsPropOptimizer,
  crossEntropy,
} from "jsr:@lala/classy@1.2.2";


We first load our dataset `iris.csv`.

In [54]:
const data = parse(Deno.readTextFileSync("../datasets/iris.csv"));

Skip the first row (header).

In [55]:
data.shift()

[
  "sepal length",
  "sepal width",
  "petal length",
  "petal width",
  "class"
]

We can now get the predictor and target variables from the dataset.

In [56]:
const x = data.map((fl, i) => fl.slice(0, 4).map(Number));

const X = new Matrix(x, "f64")
X.slice(0, 10)

0,5.1,3.5,1.4,0.2
1,4.9,3,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


In [57]:
const y_pre = data.map((fl) => fl[4]);
y_pre.slice(0, 10)

[
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa"
]

Our target variables are all strings. In order to use them for classification, we convert them into categorical variables.

In [58]:
const encoder = new CategoricalEncoder()
const y = encoder.fit(y_pre).transform(y_pre, "f64")
y.slice(0, 10)

0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [59]:
[X.shape, y.shape]

[ [ 150, 4 ], [ 150, 3 ] ]

We now split our dataset for training and testing purposes. 

In [60]:
const [[x_train, y_train], [x_test, y_test]] = useSplit(
  { ratio: [7, 3], shuffle: true },
  X,
  y
);
x_train.slice(0, 10)

0,5.1,3.5,1.4,0.2
1,4.9,3,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.8,3.4,1.6,0.2
7,4.8,3,1.4,0.1
8,5.8,4,1.2,0.2
9,5.7,4.4,1.5,0.4


Now that we have prepared our inputs, we can initialize our solver. Since we are performing logistic regression, we use a SAG solver.

We use the `crossEntropy` loss function which is used for multinomial classification, `rmsprop` as our optimizer, and finally a `softmax` function to compute joint probabilities.

In [61]:
const solver = new SagSolver({
  loss: crossEntropy(),
  activation: softmaxActivation(),
  optimizer: rmsPropOptimizer(4, 3),
});


We can then train our model using the data we acquired.

Setting the learning rate to a small value is desirable. Since our dataset is pretty simple, we are training our model for 300 epochs with 20 minibatches.

In [62]:
solver.train(x_train, y_train, {
  learning_rate: 0.01,
  epochs: 300,
  n_batches: 20,
  patience: 10
});

The model is trained, now it is time to evaluate its performance on our testing dataset

In [63]:
const res = solver.predict(x_test)
res.shape

[ 45, 3 ]

In [64]:
res.row(0)

Float64Array(3) [
  0.9999999999977868,
  2.2131906052687517e-12,
  1.5618941913383692e-23
]

The softmax function provides probabilities for the data point to belong to each of the classes. In our case, the three numbers in the array represent the probabilities of the first data point belonging to the classes `setosa`, `versicolor`, and `virginica` respectively.

We convert these into one-hot representations by taking the `argmax`.

In [65]:
CategoricalEncoder.fromSoftmax(res)
res.slice(0, 10)

0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


We can use our encoder to convert the categorical variables into class labels.

In [66]:
const y_pred = encoder.untransform(res)
const y_act = encoder.untransform(y_test)

In [67]:
[y_pred, y_act]

[
  [
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-setosa",     "Iris-versicolor",
    "Iris-versicolor", "Iris-versicolor", "Iris-versicolor",
    "Iris-versicolor", "Iris-versicolor", "Iris-versicolor",
    "Iris-versicolor", "Iris-versicolor", "Iris-virginica",
    "Iris-versicolor", "Iris-versicolor", "Iris-versicolor",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica"
  ],
  [
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-setosa",

Finally, we can generate a classification report based on our results.

In [68]:
new ClassificationReport(y_act, y_pred)

Class,Precision,F1Score,Recall,Support
Iris-setosa,1,1,1,14
Iris-versicolor,1,1,1,13
Iris-virginica,0.9473684210526315,0.972972972972973,1,18
Accuracy,,,0.9925925925925926,45


As we see, the classifier easily classifies different iris species. This is possible because the classes are easily separable. In a more complex dataset, these results may greatly vary.